In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import nltk
import keras
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from keras.models import Sequential 

Using TensorFlow backend.


In [2]:
#read data
data =pd.read_csv('data.csv')
data = data[['description','label']]
data.head(10)

,description,label
0,"♦️ I wish you to stay healthy. Your character,...",0
1,Les bases de l'algorithmique b Darija vos rema...,0
2,Corona restrictions causing terrible financial...,0
3,📣📣AMAZON recrute des conseillers clients à Dom...,1
4,Great illustration....,0
5,Great to see companies across Canada stepping ...,0
6,Le Maroc dont je suis si fier...,0
7,hashtag#عاجل : لكل من يمكنه المساعدة في تأمين ...,0
8,I just published the fourth and last blog post...,0
9,The pain you feel today is the strength you fe...,0


In [3]:

data20 = data[data.label == 1][["description"]].head(20)


In [ ]:

Stopwords = {''}
Stopwords_fr=set(stopwords.words('french'))
Stopwords_eng = set(stopwords.words('english'))
add_stop_words = "all almost also any because cause could do does done don't end for never see soon sorry than ourselves hers between yourself ut again there about once during out very having with they own an be some for do its yours such into of most itself other off is s am for who as from him each the themselves until below are we these your his through don nor me were her more himself this down should our their while above both up to ours had she all no when at any before them same and been have in will on does yourselves then that because what over why so can did not now under he you herself has just where too only myself which those i after few whom t being if theirs my against a by doing it how further was here than a à â abord afin ah ai aie ainsi allaient allo allô allons après assez attendu au aucun aucune aujourd aujourd'hui auquel aura auront aussi autre autres aux auxquelles auxquels avaient avais avait avant avec avoir ayant b bah beaucoup bien bigre boum bravo brrr c ça car ce ceci cela celle celle-ci celle-là celles celles-ci celles-là celui celui-ci celui-là cent cependant certain certaine certaines certains certes ces cet cette ceux ceux-ci ceux-là chacun chaque cher chère chères chers chez chiche chut ci cinq cinquantaine cinquante cinquantième cinquième clac clic combien comme comment compris concernant contre couic crac d da dans de debout dedans dehors delà depuis derrière des dès désormais desquelles desquels dessous dessus deux deuxième deuxièmement devant devers devra différent différente différentes différents dire divers diverse diverses dix dix-huit dixième dix-neuf dix-sept doit doivent donc dont douze douzième dring du duquel durant e effet eh elle elle-même elles elles-mêmes en encore entre envers environ es ès est et etant étaient étais était étant etc été etre être eu euh eux eux-mêmes excepté f façon fais faisaient faisant fait feront fi flac floc font g gens h ha hé hein hélas hem hep hi ho holà hop hormis hors hou houp hue hui huit huitième hum hurrah i il ils importe j je jusqu jusque k l la là laquelle las le lequel les lès lesquelles lesquels leur leurs longtemps lorsque lui lui-même m ma maint mais malgré me même mêmes merci mes mien mienne miennes miens mille mince moi moi-même moins mon moyennant n na ne néanmoins neuf neuvième ni nombreuses nombreux non nos notre nôtre nôtres nous nous-mêmes nul o o| ô oh ohé olé ollé on ont onze onzième ore ou où ouf ouias oust ouste outre p paf pan par parmi partant particulier particulière particulièrement pas passé pendant personne peu peut peuvent peux pff pfft pfut pif plein plouf plus plusieurs plutôt pouah pour pourquoi premier première premièrement près proche psitt puisque q qu quand quant quanta quant-à-soi quarante quatorze quatre quatre-vingt quatrième quatrièmement que quel quelconque quelle quelles quelque quelques quelqu'un quels qui quiconque quinze quoi quoique r revoici revoilà rien s sa sacrebleu sans sapristi sauf se seize selon sept septième sera seront ses si sien sienne siennes siens sinon six sixième soi soi-même soit soixante son sont sous stop suis suivant sur surtout t ta tac tant te té tel telle tellement telles tels tenant tes tic tien tienne tiennes tiens toc toi toi-même ton touchant toujours tous tout toute toutes treize trente très trois troisième troisièmement trop tsoin tsouin tu u un une unes uns v va vais vas vé vers via vif vifs vingt vivat vive vives vlan voici voilà vont vos votre vôtre vôtres vous vous-mêmes vu w x y z zut"
add_stop_words = add_stop_words.split()
Stopwords.update(add_stop_words)
Stopwords.update(Stopwords_fr)
Stopwords.update(Stopwords_eng)


In [5]:
data.shape

(731, 2)

In [6]:
import string
from nltk.stem import PorterStemmer

from nltk.tokenize import sent_tokenize, word_tokenize
def stemSentence(sentence):
    token_words=word_tokenize(str(sentence))
    token_words
    porter = PorterStemmer()
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)


In [7]:
data['new_description']= data['description'].apply(stemSentence)
data.head()

,description,label,new_description
0,"♦️ I wish you to stay healthy. Your character,...",0,"♦️ I wish you to stay healthi . your charact ,..."
1,Les bases de l'algorithmique b Darija vos rema...,0,le base de l'algorithmiqu b darija vo remarqu
2,Corona restrictions causing terrible financial...,0,corona restrict caus terribl financi stress to...
3,📣📣AMAZON recrute des conseillers clients à Dom...,1,📣📣amazon recrut de conseil client à domicil .🧑...
4,Great illustration....,0,great illustr ... .


In [8]:
punctuations = string.punctuation #+"’¶•@°©®™"

In [9]:
def funct(text):
    text = str(text).lower()
    
    translator = str.maketrans(punctuations," "*len(punctuations))
    s = text.translate(translator)
    
    res = ''.join([i for i in s if not i.isdigit()])
    
    wordtokens = word_tokenize(res)
    
    return  ' '.join([w for w in wordtokens if not w in Stopwords])

In [11]:
data['new_description2']=data['new_description'].apply(funct)

In [12]:
freq = pd.Series(' '.join(data['new_description2']).split()).value_counts()[:3]
freq
data['new_description2'] = data['new_description2'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))

In [13]:
data.head(15)

,description,label,new_description,new_description2
0,"♦️ I wish you to stay healthy. Your character,...",0,"♦️ I wish you to stay healthi . your charact ,...",♦️ wish stay healthi charact strength kind rest
1,Les bases de l'algorithmique b Darija vos rema...,0,le base de l'algorithmiqu b darija vo remarqu,base algorithmiqu darija vo remarqu
2,Corona restrictions causing terrible financial...,0,corona restrict caus terribl financi stress to...,corona restrict caus terribl financi stress po...
3,📣📣AMAZON recrute des conseillers clients à Dom...,1,📣📣amazon recrut de conseil client à domicil .🧑...,📣📣amazon recrut conseil client domicil 🧑‍💻proc...
4,Great illustration....,0,great illustr ... .,great illustr
5,Great to see companies across Canada stepping ...,0,great to see compani across canada step up to ...,great compani across canada step help fight co...
6,Le Maroc dont je suis si fier...,0,Le maroc dont je sui si fier ...,maroc sui fier
7,hashtag#عاجل : لكل من يمكنه المساعدة في تأمين ...,0,hashtag # عاجل : لكل من يمكنه المساعدة في تأمي...,عاجل لكل من يمكنه المساعدة في تأمين أبسط وسائل...
8,I just published the fourth and last blog post...,0,I just publish the fourth and last blog post a...,publish fourth last blog post java featur thi ...
9,The pain you feel today is the strength you fe...,0,the pain you feel today is the strength you fe...,pain feel today strength feel tomorrowhashtag ...


In [14]:
data.dropna(inplace=True)

In [16]:
x = data['new_description2']
y = data['label']

In [17]:
vectorizer = TfidfVectorizer()
x= vectorizer.fit_transform(x)


  (0, 5639)	0.31479550620807545
  (0, 3580)	0.4126273112969961
  (0, 6337)	0.3943569701348248
  (0, 1018)	0.3943569701348248
  (0, 2912)	0.3943569701348248
  (0, 6298)	0.3063150886474059
  (0, 7124)	0.4126273112969961
  (1, 5539)	0.5279037699843189
  (1, 7000)	0.3090066386266902
  (1, 1613)	0.5279037699843189
  (1, 241)	0.5279037699843189
  (1, 650)	0.2616633833643219
  (2, 1669)	0.16062438938320386
  (2, 161)	0.15118920975531885
  (2, 3886)	0.10379093434224992
  (2, 7038)	0.16062438938320386
  (2, 2811)	0.16062438938320386
  (2, 2357)	0.12095487368649681
  (2, 2998)	0.11373756348519617
  (2, 4587)	0.16062438938320386
  (2, 5329)	0.1444948432734052
  (2, 4377)	0.08393021217015281
  (2, 6042)	0.16062438938320386
  (2, 4413)	0.1444948432734052
  (2, 2747)	0.12836529716360653
  :	:
  (730, 3424)	0.04240867020843893
  (730, 3427)	0.04240867020843893
  (730, 3431)	0.04240867020843893
  (730, 1721)	0.08902020306892465
  (730, 6364)	0.058356818932424025
  (730, 4993)	0.056169088009404795
  (7

# save Tfidf

In [23]:
import pickle
#save tfidf 
pickle.dump(vectorizer,open("tfidf.pkl","wb"))

In [24]:
x.shape

<class 'scipy.sparse.csr.csr_matrix'>


In [83]:
from sklearn.model_selection import train_test_split

In [84]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=60)

In [85]:
from sklearn.svm import SVC

In [86]:
X_train.shape

(976, 10040)

In [87]:
X_test.shape

(245, 10040)

In [88]:
clf = SVC()
clf.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [90]:
pr=clf.predict(X_test)

In [91]:
testprecision = accuracy_score(pr, Y_test)
print("test precision = {}%".format(round(testprecision * 100),1))

test precision = 91.0%


# save model

In [108]:
my_model = 'final_model.pkl'
with open(my_model, 'wb') as file:  
    pickle.dump(clf, file)

# testing the model

In [106]:
data20['description'] = data20['description'].apply(stemSentence)
data20['description'] = data20['description'].apply(funct)

tfidf_vectorizer = pickle.load(open("tfidf.pkl", "rb"))
new_dsc = tfidf_vectorizer.transform(data20['description'])
print(type(data20['description']))

my_model = 'final_model.pkl'
with open(my_model, 'rb') as f:
    model = pickle.load(f)
    
model.predict(new_dsc)

<class 'pandas.core.series.Series'>


array([1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0],
      dtype=int64)